---
Здесь будет такая идея: стейты для всевозможных пар из N частиц делаем: N(N - 1) / 2 их штук - потом проходимся циклом по всем этим стейтам и делаем для каждого предсказание

In [1]:
# states = {
    # particle_number: [lists of states with this particle]
# }